# Notebook to Segment Skeletal Muscle Area from MRI images (TIFs)

###### Load libraries and directories

In [22]:
import os
os.getcwd()
os.chdir('/tf/smipipeline')
print(os.getcwd())

/tf/smipipeline


In [23]:
from IPython.display import display, HTML

In [24]:
# from IPython import get_ipython
from tqdm.notebook import tqdm
import pickle
import os
import pprint
import numpy as np

import tables
from MRI_notebooks.TIF_loader import mri_TIF_loader
from unet3d.normalize import normalize_data_storage_2D


pp = pprint.PrettyPrinter(indent=1)
 
import json
import pprint

# Custom functions
import pickle
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

def load_object(filename):        
    with open(filename, 'rb') as input:
        return pickle.load(input)

In [25]:
cwd = os.getcwd()
print(cwd)
data = '/tf/data'
pickles = '/tf/pickles'
models = '/tf/models'

/tf/smipipeline


In [27]:
# Import modules and config file
configfile = os.path.join(cwd,'config/mri/run_preproc_mri.json')
with open(configfile, "r") as f:
        config = json.load(f)
pp.pprint(config)

{'l3_finder': {'cache_dir': '/tf/_cache/',
               'cache_intermediate_results': True,
               'model_path_dir': '/tf/models/l3/cv_final',
               'new_tim_dicom_dir_structure': True,
               'output_directory': '/tf/output/cv_poorl3/l3',
               'overwrite': True,
               'save_plots': True,
               'show_plots': False,
               'tif_dir': '/tf/data/sarcopeniaMR_L3_TIF'},
 'muscle_segmentor': {'input_directory': '/tf/data/sarcopeniaMR_L3_TIF',
                      'output_directory': '/tf/data/sarcopeniaMR_L3_h5',
                      'sma_config_args_file': '/tf/smipipeline/config/mri/sma_mri_preproc.args'}}


In [28]:
## Preprocess config to convert tif to hdf5 file for training
config = config['muscle_segmentor']
##
config["input_type"] = "Image"
config["input_shape"] = (256,256)
config["input_images"] = "/tf/data" #config['input_directory']
config["image_format"] = "TIF" # or "NIFTI"
config["slice_number"] = 0 # Use this if you have a stacked TIF and want only one slice for 2D problems.
                           # slice number goes from 0 to length of Stack
config['use_middle_image'] = False

config["output_file"] = "combined_aug3_205_fixed.h5"
config["output_directory"] = '/tf/data/combined_aug3_205'

config["overwrite"] = 1
config["problem_type"] = "Segmentation"
config["image_modalities"] = ["CT"]
config["image_masks"] = ["MASK" ] #["Label"]   # For Image Masks, will serve as label for segmentation problems
config["n_channels"] = 0            # All image channels that will be used as input, image_mask can be input for classification problems and output for segmentation problems.

config["clinical_truthname"] =  None # For CSV File
config["normalize"] = True
config['normalize_on_load'] = True
config["invert_image"] = True

In [29]:
pp.pprint(config)

{'clinical_truthname': None,
 'image_format': 'TIF',
 'image_masks': ['MASK'],
 'image_modalities': ['CT'],
 'input_directory': '/tf/data/sarcopeniaMR_L3_TIF',
 'input_images': '/tf/data',
 'input_shape': (256, 256),
 'input_type': 'Image',
 'invert_image': True,
 'n_channels': 0,
 'normalize': True,
 'normalize_on_load': True,
 'output_directory': '/tf/data/combined_aug3_205',
 'output_file': 'combined_aug3_205_fixed.h5',
 'overwrite': 1,
 'problem_type': 'Segmentation',
 'slice_number': 0,
 'sma_config_args_file': '/tf/smipipeline/config/mri/sma_mri_preproc.args',
 'use_middle_image': False}


### Set overwrite to false when extracting imags from h5 file

In [30]:
config['overwrite'] = False

In [31]:
def get_image_loader(config,problem_type,input_images):
    return mri_TIF_loader(
                problem_type,
                input_images,
                config["input_shape"],
                config["image_modalities"],
                config["image_masks"],
                config['slice_number'],config['invert_image'],config['normalize_on_load'])

In [32]:
# Step 1: Check if Input Folders are defined
try:
    input_type = config["input_type"]
except:
    print("Error: Input type for preprocessing not defined | \t Set  config[\"input_type\"] to \"Image\", \"Clinical\" or \"Both\" \n")
    
input_images = config["input_images"]
input_clinical = None
if (input_type == "Image" or input_type == "Both"):
    try:
        input_images = os.path.abspath(config["input_images"])
    except:
        print("Error: Input Image Folder for preprocessing not defined | \t Set config[\"input_images\"] \n")

if (input_type == "Clinical" or input_type == "Both"):
    try:
        input_clinical = os.path.abspath(config["input_clinical"])
    except:
        print("Error: Input Clinical Folder with .csv for preprocessing not defined | \t Set config[\"input_clinical\"] \n")

In [33]:
# Step 2: Check if the Output File is defined
try:
    output_file = os.path.abspath(os.path.join(config["output_directory"], config["output_file"]))
except:
    print("Error: Input type for preprocessing not defined | \t Set  config[\"input_type\"] to \"Image\",\"Clinical\" or \"Both\" \n")

In [34]:
# Step 3: Check if Output file already exists, If it exists, require user permission to overwrite
if 'overwrite' in config:
    overwrite = config["overwrite"]
elif os.path.exists(output_file):
    overwrite = input("Output file exists, do you want to overwrite? (y/n) \n")
    overwrite = True if overwrite == 'y' else False   

    # Open the hdf5 file
if overwrite:
    print("Opening h5 file in Write mode...")
    hdf5_file = tables.open_file(output_file, mode='w')
else:
    print("Opening h5 file in Read mode...")
    hdf5_file = tables.open_file(output_file, mode='r')

Opening h5 file in Read mode...


In [35]:
# Step 4: Check problem specific parameters are defined
problem_type = config['problem_type']
print('Problem type is: ', problem_type)
print('Input Type is: ', input_type)
print('Input Images is: ', input_images)

Problem type is:  Segmentation
Input Type is:  Image
Input Images is:  /tf/data


In [36]:
# Step 5: Convert Exams to hdf5 
#  Load Imaging Data
image_loader = get_image_loader(config,problem_type,input_images)

In [37]:
print(image_loader)

In [38]:
subject_ids_final = image_loader.get_sample_ids()
print("Total number of subjects in h5 file", len(subject_ids_final))

Total number of subjects in h5 file 7


In [39]:
training_modalities = config['image_modalities']
data_file =hdf5_file

In [40]:
from PIL import Image
import numpy as np

In [41]:
for data_index,subject in enumerate(subject_ids_final):
    output_dir = os.path.join(config['output_directory'],subject)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    test_data = np.asarray([data_file.root.imdata[data_index]])
    for i, modality in enumerate(training_modalities):
        img = test_data[0,i]
        image = Image.fromarray(test_data[0, i])
        image.save(os.path.join(output_dir, "data_{0}.TIF".format(modality)))

    truth = data_file.root.truth[data_index][0].astype(np.float32)
    test_truth = Image.fromarray(truth)
    test_truth.save(os.path.join(output_dir, "truth.TIF"))

In [42]:
hdf5_file.close()